# 🔍 Construction de l'Index FAISS

Création de l'index vectoriel FAISS pour la recherche sémantique rapide.

**Étapes :**

1. Chargement des embeddings
2. Construction de l'index FAISS
3. Test de recherche
4. Export de l'index


## 1️⃣ Configuration


In [4]:
import json
import numpy as np
from pathlib import Path

import faiss
from sentence_transformers import SentenceTransformer

from src.config.constants import PROCESSED_DATA_DIR

print("✅ Configuration OK")

✅ Configuration OK


## 2️⃣ Chargement des Données


In [5]:
embeddings_dir = PROCESSED_DATA_DIR / "embeddings"

# Charger embeddings
embeddings_path = embeddings_dir / "embeddings.npy"
embeddings = np.load(embeddings_path)

# Charger métadonnées
metadata_path = embeddings_dir / "metadata.json"
with open(metadata_path, "r", encoding="utf-8") as f:
    metadata = json.load(f)

# Charger documents
documents_path = PROCESSED_DATA_DIR / "rag_documents.json"
with open(documents_path, "r", encoding="utf-8") as f:
    documents = json.load(f)

print(f"📊 {len(embeddings)} embeddings chargés")
print(f"📐 Dimension : {embeddings.shape[1]}")
print(f"📚 {len(documents)} documents chargés")

📊 497 embeddings chargés
📐 Dimension : 1024
📚 497 documents chargés


## 3️⃣ Construction de l'Index FAISS

Utilisation de **IndexFlatL2** pour une recherche exacte.

**Dimensions supportées :**

- Mistral : 1024
- SentenceTransformers : 768 (ou variable selon le modèle)


In [6]:
# Dimension des embeddings
dimension = embeddings.shape[1]

# Créer l'index FAISS
# IndexFlatL2 : recherche exacte par distance L2
index = faiss.IndexFlatL2(dimension)

# Normaliser pour similarité cosinus (optionnel)
faiss.normalize_L2(embeddings)

# Ajouter les vecteurs à l'index
index.add(embeddings)

print(f"✅ Index FAISS créé")
print(f"📊 Nombre de vecteurs : {index.ntotal}")
print(f"📐 Dimension : {index.d}")

✅ Index FAISS créé
📊 Nombre de vecteurs : 497
📐 Dimension : 1024


## 4️⃣ Test de Recherche


In [7]:
from src.config.settings import settings

In [8]:
# Classe d'encodage unifiée (copie du notebook 03)
from mistralai import Mistral
from sentence_transformers import SentenceTransformer


class QueryEncoder:
    """Encodeur de requêtes unifié."""

    def __init__(self, provider: str, model_name: str):
        self.provider = provider
        self.model_name = model_name

        print(f"🔄 Chargement : provider={provider}, model={model_name}")

        if provider == "mistral":
            self.client = Mistral(api_key=settings.mistral_api_key)
        else:
            self.model = SentenceTransformer(model_name)

        print("✅ Modèle chargé")

    def encode(self, text: str) -> np.ndarray:
        """Encode un texte en vecteur."""
        if self.provider == "mistral":
            response = self.client.embeddings.create(model=self.model_name, inputs=[text])
            return np.array([response.data[0].embedding], dtype=np.float32)
        else:
            return self.model.encode([text], convert_to_numpy=True)


# Charger l'encodeur avec le bon provider
provider = metadata.get("provider", "sentence-transformers")
model_name = metadata["model_name"]

encoder = QueryEncoder(provider, model_name)

🔄 Chargement : provider=mistral, model=mistral-embed
✅ Modèle chargé


In [9]:
def search_events(query: str, top_k: int = 5):
    """Recherche les événements les plus similaires à la requête."""

    # Encoder la requête avec le bon provider
    query_embedding = encoder.encode(query)
    faiss.normalize_L2(query_embedding)

    # Recherche dans l'index
    distances, indices = index.search(query_embedding, top_k)

    # Retourner les résultats
    results = []
    for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
        doc = documents[idx]
        results.append(
            {
                "rank": i + 1,
                "distance": float(dist),
                "similarity": float(1 - dist),  # Conversion distance -> similarité
                "title": doc["title"],
                "city": doc["metadata"]["city"],
                "url": doc["metadata"]["url"],
                "content": doc["content"][:200] + "...",
            }
        )

    return results


print("✅ Fonction de recherche définie")

✅ Fonction de recherche définie


In [10]:
# Test 1 : Recherche de concerts
print("🔍 Test 1 : 'concert de musique classique'\n")
results = search_events("concert de musique classique", top_k=3)

for result in results:
    print(f"#{result['rank']} - Similarité: {result['similarity']:.3f}")
    print(f"   Titre: {result['title']}")
    print(f"   Ville: {result['city']}")
    print()

🔍 Test 1 : 'concert de musique classique'

#1 - Similarité: 0.468
   Titre: Rouvrir le monde : Documentaire à la maison de la musique des 15-16 « Notre enfance »
   Ville: Marseille

#2 - Similarité: 0.448
   Titre: MÉLODIES ET MYSTÈRES DES HAUTS LIEUX DE LA MUSIQUE
   Ville: Marseille

#3 - Similarité: 0.442
   Titre: Pauses musicales
   Ville: Marseille



In [11]:
# Test 2 : Recherche d'expositions
print("🔍 Test 2 : 'exposition art contemporain'\n")
results = search_events("exposition art contemporain", top_k=3)

for result in results:
    print(f"#{result['rank']} - Similarité: {result['similarity']:.3f}")
    print(f"   Titre: {result['title']}")
    print(f"   Ville: {result['city']}")
    print()

🔍 Test 2 : 'exposition art contemporain'

#1 - Similarité: 0.531
   Titre: Exposition d'art contemporain
   Ville: Marseille

#2 - Similarité: 0.490
   Titre: Exposition
   Ville: Marseille

#3 - Similarité: 0.449
   Titre: Exposition "Hors Site (mais pas hors sol)"
   Ville: Marseille



In [12]:
# Test 3 : Recherche par ville
print("🔍 Test 3 : 'événement à Marseille'\n")
results = search_events("événement à Marseille", top_k=3)

for result in results:
    print(f"#{result['rank']} - Similarité: {result['similarity']:.3f}")
    print(f"   Titre: {result['title']}")
    print(f"   Ville: {result['city']}")
    print()

🔍 Test 3 : 'événement à Marseille'

#1 - Similarité: 0.599
   Titre: "Si Victor m'était conté"
   Ville: Marseille

#2 - Similarité: 0.587
   Titre: Rendez-vous aux jardins de la Citadelle de Marseille
   Ville: Marseille

#3 - Similarité: 0.586
   Titre: EXPOSITION du concours d'idée "Marseille ville récréative"
   Ville: Marseille



## 5️⃣ Statistiques de Performance


In [13]:
import time

# Benchmark de vitesse
test_queries = [
    "concert jazz",
    "théâtre enfants",
    "festival musique",
    "exposition peinture",
    "atelier créatif",
]

times = []
for query in test_queries:
    start = time.time()
    results = search_events(query, top_k=10)
    elapsed = time.time() - start
    times.append(elapsed)

avg_time = sum(times) / len(times)

print("⚡ Performance de recherche :")
print(f"  Temps moyen : {avg_time * 1000:.2f} ms")
print(f"  Recherches/sec : {1 / avg_time:.2f}")

⚡ Performance de recherche :
  Temps moyen : 446.40 ms
  Recherches/sec : 2.24


## 6️⃣ Export de l'Index


In [2]:
from src.config.constants import PROCESSED_DATA_DIR


In [14]:
# Créer le dossier index
index_dir = PROCESSED_DATA_DIR / "faiss_index"
index_dir.mkdir(exist_ok=True)

# Export index FAISS
index_path = index_dir / "events.index"
faiss.write_index(index, str(index_path))

# Export configuration
config_path = index_dir / "config.json"
config = {
    "model_name": model_name,
    "index_type": "IndexFlatL2",
    "dimension": dimension,
    "num_vectors": index.ntotal,
    "normalized": True,
    "documents_path": str(documents_path),
}

with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print(f"💾 Index FAISS : {index_path}")
print(f"💾 Configuration : {config_path}")
print(f"\n✅ Export terminé !")
print(f"\n➡️ Prochaine étape : Notebook 05 - Test du système RAG complet")

💾 Index FAISS : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/processed/faiss_index/events.index
💾 Configuration : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/processed/faiss_index/config.json

✅ Export terminé !

➡️ Prochaine étape : Notebook 05 - Test du système RAG complet
